# UE4 - Tonemapping Operator

In [1]:
from __future__ import division, unicode_literals

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from collections import namedtuple

from ipywidgets import interact

BOKEH_TOOLS = 'pan,wheel_zoom,box_zoom,reset,resize,hover'

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def lerp(a, b, c):
    return (1 - c) * a + c * b


def tonemapping_operator_unreal_engine_4(
    a, 
    film_slope=0.91, 
    film_toe=0.53, 
    film_shoulder=0.23, 
    film_black_clip=0, 
    film_white_clip=0.035, 
    in_match=0.18, 
    out_match=0.18):

    toe_scale = 1 + film_black_clip - film_toe
    shoulder_scale = 1 + film_white_clip - film_shoulder

    if film_toe > 0.8:
        toe_match = (1 - film_toe - out_match) / film_slope + np.log10(in_match)
    else:
        bt = (out_match + film_black_clip) / toe_scale - 1
        toe_match = np.log10(in_match) - 0.5 * np.log((1 + bt) / (1 - bt)) * (
            toe_scale / film_slope)

    straight_match = (1 - film_toe) / film_slope - toe_match
    shoulder_match = film_shoulder / film_slope - straight_match

    log_colour = np.log10(a)
    straight_colour = film_slope * (log_colour + straight_match)

    toe_colour = (-film_black_clip) + (2 * toe_scale) / (
        1 + np.exp((-2 * film_slope / toe_scale) * (log_colour - toe_match)))
    shoulder_colour = (1 + film_white_clip) - (2 * shoulder_scale) / (
        1 + np.exp((2 * film_slope / shoulder_scale) * (log_colour - shoulder_match)))

    toe_colour = np.where(log_colour < toe_match, toe_colour, straight_colour)
    shoulder_colour = np.where(log_colour > shoulder_match, shoulder_colour,
                               straight_colour)

    t = np.clip((log_colour - toe_match) / (shoulder_match - toe_match), 0, 1)
    t = np.where(shoulder_match < toe_match, 1 - t, t)
    t = (3 - 2 * t) * t * t
    tone_colour = lerp(toe_colour, shoulder_colour, t)
    
    return tone_colour

In [4]:
MINIMUM_REPR_NUMBER = 0.18 * 2 ** -15
# MAXIMUM_REPR_NUMBER = 0.18 * 2 ** 18
MAXIMUM_REPR_NUMBER = 1

DEFAULT_SAMPLE_COUNT = 1024

SAMPLES_LOG = np.logspace(
    np.log10(MINIMUM_REPR_NUMBER), 
    np.log10(MAXIMUM_REPR_NUMBER), 
    DEFAULT_SAMPLE_COUNT)
SAMPLES = np.linspace(
    SAMPLES_LOG[0], 
    SAMPLES_LOG[-1], 
    DEFAULT_SAMPLE_COUNT / 4)

FILM_SLOPE=0.91
FILM_TOE=0.53
FILM_SHOULDER=0.23
FILM_BLACK_CLIP=0
FILM_WHITE_CLIP=0.035
IN_MATCH = 0.18
OUT_MATCH=0.18


def tonemapping_operator_unreal_engine_4_interactive(
    film_slope=FILM_SLOPE, 
    film_toe=FILM_TOE, 
    film_shoulder=FILM_SHOULDER, 
    film_black_clip=FILM_BLACK_CLIP, 
    film_white_clip=FILM_WHITE_CLIP, 
    in_match=IN_MATCH, 
    out_match=OUT_MATCH):
    
    plot = figure(tools=BOKEH_TOOLS, x_range=(0, 1), y_range=(0, 1))

    ue4_line = plot.line(
        SAMPLES, 
        tonemapping_operator_unreal_engine_4(
            SAMPLES,
            film_slope,
            film_toe,
            film_shoulder,
            film_black_clip,
            film_white_clip,
            in_match,
            out_match), 
        color='green', 
        legend='UE4 Tonemapping Operator')
    plot.legend.location = 'top_left'
    
    handle = show(plot, notebook_handle=True)
    
    return {'ue4_line': ue4_line, 'handle': handle}


UE4_t_o_i = tonemapping_operator_unreal_engine_4_interactive(
    film_slope=FILM_SLOPE,
    film_toe=FILM_TOE,
    film_shoulder=FILM_SHOULDER,
    film_black_clip=FILM_BLACK_CLIP,
    film_white_clip=FILM_WHITE_CLIP,
    in_match=IN_MATCH,
    out_match=OUT_MATCH)


def update_UE4_t_o_i(
    film_slope=FILM_SLOPE,
    film_toe=FILM_TOE,
    film_shoulder=FILM_SHOULDER,
    film_black_clip=FILM_BLACK_CLIP,
    film_white_clip=FILM_WHITE_CLIP,
    in_match=IN_MATCH,
    out_match=OUT_MATCH):
    UE4_t_o_i['ue4_line'].data_source.data['y'] = tonemapping_operator_unreal_engine_4(
            SAMPLES,
            film_slope,
            film_toe,
            film_shoulder,
            film_black_clip,
            film_white_clip,
            in_match,
            out_match)

    push_notebook(handle=UE4_t_o_i['handle'])  

In [5]:
interact(update_UE4_t_o_i, film_slope=(0, 2, 0.001), film_toe=(0, 2, 0.001), film_shoulder=(0, 2, 0.001), film_black_clip=(0, 2, 0.001), film_white_clip=(0, 2, 0.001), in_match=(0, 2, 0.001), out_match=(0, 2, 0.001))